# Implement one-vs-all logistic regression and neural networks to recognize hand-written digits.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [3]:
data  = loadmat('/content/ex4data1.mat')

In [4]:
data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [5]:
X = data['X']
y = data['y']

X.shape, y.shape ## look at the dimensions

((5000, 400), (5000, 1))

We also need to perform one-hot encoding on our y tag. The one-hot encoding converts the class label n (k class) into a vector of length k, where the index n is "hot" (1), and the rest are 0. Scikitlearn has a built-in utility, we can use this.


In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [7]:
y[0], y_onehot[0,:]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

The neural network we are going to build for this exercise has an input layer that matches the size of our instance data (400 + bias units), a hidden layer of 25 units (26 with bias units), and an output layer, 10 units correspond to one of our one-hot encoding labels. 

The first thing we need to achieve is to evaluate the cost function of the loss of a given set of network parameters. 

In [8]:
## Here we use Sigmoid Function
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

In [9]:

## creating a forward Propagate function
def forward_propagate(X, val1, val2):
    m = X.shape[0]
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * val1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * val2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h


In [10]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    val1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    val2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, val1, val2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    return J


We have used this Sigmoid function before. The forward propagation function calculates the hypothesis for each training instance given the current parameters. Its output shape should be the same as a one-hot encoding of y.

In [11]:
# Initialize settings
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 1

# Randomly initialize the parameter array of the complete network parameter size.
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)
# Unpack the parameter array into the parameter matrix of each layer
val1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
val2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

val1.shape, val2.shape

((25, 401), (10, 26))

In [12]:
a1, z2, a2, z3, h = forward_propagate(X, val1, val2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

Cost function After calculating the hypothesis matrix h, the cost function is applied to calculate the total error between y and h.


In [13]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)


6.687350525309224

Next is the back propagation algorithm. The backpropagation parameter update calculation will reduce the network error on the training data. The first thing we need is a function to calculate the gradient of the Sigmoid function we created earlier.


In [14]:
def sigmoid_gradient(z):
  return np.multiply(sigmoid(z), (1 - sigmoid(z)))

Now we are ready to implement backpropagation to calculate the gradient. Since the calculation required for backpropagation is the calculation process required in the cost function, we will actually extend the cost function to perform backpropagation and return the cost and gradient.


In [15]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    val1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    val2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, val1, val2)
    
    # initializations
    J = 0
    delta1 = np.zeros(val1.shape)  # (25, 401)
    delta2 = np.zeros(val2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(val1[:,1:], 2)) + np.sum(np.power(val2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((val2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

The hardest part of backpropagation calculations (other than understanding why we are doing all these calculations) is to get the correct matrix dimensions. By the way, you can easily confuse the use of A * B with np.multiply(A, B). Basically the former is matrix multiplication and the latter is element-wise multiplication (unless A or B is a scalar value, in which case it doesn't matter). Anyway, let's test it to make sure the function returns what we expect.


In [16]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape


(6.692664026112636, (10285,))

We also need to make a modification to the back propagation function, that is, adding regularization to the gradient calculation. The final official version is as follows.




In [17]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    val1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    val2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, val1, val2)
    
    # initializations
    J = 0
    delta1 = np.zeros(val1.shape)  # (25, 401)
    delta2 = np.zeros(val2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(val1[:,1:], 2)) + np.sum(np.power(val2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((val2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (val1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (val2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [18]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape


(6.692664026112636, (10285,))

We are finally ready to train our network and use it to make predictions.

In [19]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply


     fun: 0.3775962670550885
     jac: array([ 2.74217663e-05, -6.01173651e-06,  1.66093658e-06, ...,
       -1.66378210e-04, -6.04143793e-04, -2.36458217e-04])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 19
  status: 3
 success: False
       x: array([-0.08399857, -0.03005868,  0.00830468, ...,  0.69877635,
        1.79638938, -0.98605801])

Since the objective function is unlikely to converge completely, we limit the number of iterations. Our total cost has fallen below 0.5, which is a good indicator that the algorithm is working properly. Let's use the parameters it finds and forward it through the network to get some predictions.


Let's use the parameters it finds and propagate it forward through the network to obtain predictions.


In [20]:
X = np.matrix(X)
val1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
val2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, val1, val2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred


array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]])

Finally, we can calculate the accuracy and see how well the neural network we have trained performs.


In [21]:
correct = [1 if a == b else 0 for (a,b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 98.28%


We have successfully implemented a basic backpropagation neural network and used it to classify handwritten digital images. 